<a href="https://colab.research.google.com/github/SamaSamrin/DL-Final-Project/blob/main/DL_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Loading dataset**

In [3]:
import numpy as np
from six.moves import urllib
import os
import tensorflow as tf
import pickle

DATA_DIRECTORY = "data"
SOURCE_URL = 'http://35.183.27.27/'

def maybe_download(filename):
    """Download the data, unless it's already here."""
    if not tf.io.gfile.exists(DATA_DIRECTORY):
        tf.gfile.MakeDirs(DATA_DIRECTORY)
    filepath = os.path.join(DATA_DIRECTORY, filename)     
    if not tf.io.gfile.exists(filepath):
        print('Start downloading dataset...')
        filepath, _ = urllib.request.urlretrieve(SOURCE_URL + filename, filepath)         
        with tf.gfile.GFile(filepath) as f:
            size = f.size()
        print('Successfully downloaded', filename, size, 'bytes.')
    return filepath

def dense_to_one_hot(labels_dense, num_classes = 3):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

def load_data(filename, one_hot=False):
    with open(filename, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
        data = dict['data']
        labels = dict['label']

        if one_hot:
            labels = dense_to_one_hot(labels)
        return data, labels

class DataSet(object):
    def __init__(self, data, labels):
        assert data.shape[0] == labels.shape[0], ("data.shape: %s labels.shape: %s" % (data.shape, labels.shape))
        self._num_examples = data.shape[0]

        self._data = data
        self._lables = labels
        self._epochs_completed = 0
        self._index_in_epoch = 0

    @property
    def data(self):
        return self._data

    @property
    def labels(self):
        return self._lables

    @property
    def num_examples(self):
        return self._num_examples

    @property
    def epochs_completed(self):
        return self._epochs_completed

    def next_batch(self, batch_size):
        start = self._index_in_epoch
        self._index_in_epoch += batch_size
        if self._index_in_epoch > self._num_examples:
            self._epochs_completed += 1
            perm = np.arange(self._num_examples)
            np.random.shuffle(perm)
            self._data = self._data[perm]
            self._lables = self._lables[perm]

            start = 0

            self._index_in_epoch = batch_size

            assert batch_size <= self._num_examples
        end = self._index_in_epoch
        return self._data[start:end], self._lables[start:end]

def read_data_sets(one_hot=False):
    class DataSets(object):
        pass
    train_filename = maybe_download('train')
    test_filename = maybe_download('test')

    train_data, train_labels = load_data(train_filename, one_hot)
    test_data, test_labels = load_data(test_filename, one_hot)
    data_sets = DataSets()
    data_sets.train = DataSet(train_data, train_labels)
    data_sets.test = DataSet(test_data, test_labels)
    return data_sets

In [6]:
# return DataSet class
data = read_data_sets(one_hot=True)

# get train data and labels by batch size
train_x, train_label = data.train.next_batch(500)

# get test data
test_x = data.test.data

# get test labels
test_labels = data.test.labels

# get sample number
n_samples = data.train.num_examples

In [7]:
print(train_x)

[[26.94933313 26.94942352 26.86116345 ... 16.22032493 17.53295088
  17.53075379]
 [26.95540644 26.95561349 26.86818211 ... 16.22077216 17.53405894
  17.53186146]
 [26.96394591 26.96354065 26.87468671 ... 16.22162589 17.53570343
  17.53347345]
 ...
 [26.56521813 26.37605141 26.65220509 ... 16.86037896 18.29403388
  18.29335327]
 [26.54994368 26.35584233 26.64187862 ... 16.85800073 18.29309627
  18.29231912]
 [26.53391519 26.3387579  26.63412098 ... 16.85556478 18.29161395
  18.2907373 ]]


# **Next Step**